In [4]:
import importlib

In [1]:
import os

---

# CORUM

**2015 November 29-30**

Processing the downloaded CORUM file *CORUM_allComplexes.csv* into 2-column format (1<sup>st</sup> column &larr; complex name, 2<sup>nd</sup> column &larr; gene) with the script *proteincomplexformat.py*. The file lists complex members as UniProt IDs and ostensibly Entrez IDs. Unfortunately, it will not be possible to directly process the CORUM file into Entrez IDs. For whatever reason, CORUM maps some UniProt IDs to Entrez ID 0:

In [2]:
%%bash
grep ';SRp30c-SRp55 complex' /work/jyoung/DataDownload/CORUM_allComplexes.csv

6073;SRp30c-SRp55 complex;;Human;Q13247,Q13242;0,0;MI:0019- coimmunoprecipitation;15695522;11.04.03.01.10;"";"Tau exons 2 and 10, which are misregulated in neurodegenerative diseases, are partly regulated by silencers which bind a SRp30c-SRp55 complex that either recruits or antagonizes htra2beta1 (a variant of TRA2B).";""


<s>Plan to use the *corum()* function in *proteincomplexformat.py* to write IDs as UniProt and the Bioconductor R *org.Hs.eg.db* library to convert these to Entrez.</s> Unfortunately, *org.Hs.eg.db* only converts from Entrez to UniProt and not the other way around. Try to write out all the UniProt IDs from CORUM into a single file and have the [UniProt mapping service](http://www.uniprot.org/mapping/) perform the conversion. Also beware that for some reason, the CORUM file has parentheses surrounding some of the IDs:

In [3]:
%%bash
grep ';NK-3-Groucho complex;' /work/jyoung/DataDownload/CORUM_allComplexes.csv

3150;NK-3-Groucho complex;;Human;(Q99801,P78367),(Q04724,Q04725,Q04726,Q04727,Q9H808);(4824,579),(7088,7089,7090,7091,79816);MI:0096- pull down;10559189;11.02.03.04.03,70.10;"NK-3 homeodomain protein can associate wih the human Groucho homolog TLE in the absence of DNA. This interaction translocates Groucho proteins from the cytoplasm into the nucleus.";"";""


In [4]:
os.chdir(os.path.join(os.sep, 'work', 'jyoung','DataDownload'))
uniprotIDs = set()
corumFile = open('CORUM_allComplexes.csv')
header = corumFile.readline().rstrip().split(';')
orgCol = header.index('organism')
uniprotCol = header.index('subunits (UniProt IDs)')
for line in corumFile:
    tokens = line.rstrip().split(';')
    if tokens[orgCol] == 'Human':
        if tokens[uniprotCol] != '':
            if '(' in tokens[uniprotCol]:
                uniprotIDs.update(tokens[uniprotCol].replace('(', '').replace(')', '').split(','))
            else:
                uniprotIDs.update(tokens[uniprotCol].split(','))
corumFile.close()

os.chdir(os.path.join('..', 'DataProcessed'))
writeFile = open('CORUM_Human_UniProt.txt', 'w')
for uid in uniprotIDs:
    writeFile.write(uid + '\n')
writeFile.close()

2,539 out of 2,555 identifiers from UniProtKB AC/ID were successfully mapped to 2,576 Entrez Gene (GeneID) IDs. The mapped IDs were saved as *CORUM_Hs_UniProt_mapped.tab* and the un-mapped IDs were saved into *CORUM_Hs_UniProt_notmapped.txt*. Any un-mapped IDs will be ignored when converting from UniProt to Entrez. The mapping is also not 1-to-1:

In [5]:
%%bash
wc -l /work/jyoung/DataProcessed/CORUM_Hs_UniProt_mapped.tab

2583 /work/jyoung/DataProcessed/CORUM_Hs_UniProt_mapped.tab


In [6]:
%%bash
cut -f1 /work/jyoung/DataProcessed/CORUM_Hs_UniProt_mapped.tab | sort | uniq | wc -l

2540


In such instances, the offending UniProt ID with multiple corresponding Entrez IDs will be ignored and not converted.

Finally, the DNMT3L-DNMT3A complex entry in the download CORUM file has extra spaces in the complex name that cause a tab to be detected in the name when splitting on tab-delimited spaces:

In [1]:
%%bash
grep 'DNMT3L-DNMT3A' /work/jyoung/DataDownload/CORUM_allComplexes.csv

2651;DNMT3L-DNMT3A	complex;;Human;Q9Y6K1,Q9UJW3;1788,29947;MI:0071- molecular sieving;15105426;10.01.09.01;"";"";""


Manually correct this in the *CORUM_allComplexes.csv* file.

Modifications made to *proteincomplexformat.py* to add a function to return a dictionary convert UniProt to Entrez and perform the processing to CORUM to 2-column format.

In [1]:
import proteincomplexformat

In [2]:
proteincomplexformat.corum('Human')

2-column output written as *CORUM_Human_Entrez.txt*. 

It turns out that the download CORUM file also contains duplicate entries:

In [1]:
%%bash
grep ';P-TEFb-7SKRNA-HEXIM1 complex;' /work/jyoung/DataDownload/CORUM_allComplexes.csv

2602;P-TEFb-7SKRNA-HEXIM1 complex;;Human;O60563,P50750,O94992;904,1025,10614;MI:0007- anti tag coimmunoprecipitation;14580347;01.04,11.02.03.01.04,11.02.03.04,18.01.01,18.02.01,70.10;"7SK RNA is required for HEXIM1 to associate with P-TEFb complex. This allowes HEXIM1 to inhibit transcription. P-TEFb dissociates from HEXIM1 and 7SK in cells undergoing stress response, increasing the level of active P-TEFb for stress-induced transcription.";"";""
2873;P-TEFb-7SKRNA-HEXIM1 complex;;Human;O60563,P50750,O94992;904,1025,10614;MI:0071- molecular sieving | MI:0007- anti tag coimmunoprecipitation;12832472;01.04,11.02.03.01.04,11.02.03.04,18.01.01,18.02.01,70.10;"7SK RNA is required for MAQ1 to associate with P-TEFb complex. P-TEFb is inactive when bound to MAQ1 and 7SK. The 7SK RNA/MAQ1 system appears to be a new kind of CDK regulator that may contribute to a feedback loop modulating the activity of RNAP II.";"";""
2878;P-TEFb-7SKRNA-HEXIM1 complex;;Human;O60563,P50750,O94992;904,1025,10614;MI

Remove duplicate lines from *CORUM_Human_Entrez.txt*:

In [3]:
%%bash
cd /work/jyoung/DataProcessed/
mv CORUM_Human_Entrez.txt CORUM_Human_Entrez.old
awk '!seen[$0]++' CORUM_Human_Entrez.old > CORUM_Human_Entrez.txt